In [247]:
import sys
sys.path.append('../')
from utility.genotype_file_utility import GenotypeReader
from utility.file_utility import FileUtility
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import tqdm

In [248]:

def getIDXs(isolates,df):
    isolates_all=df['Isolates'].values.tolist()
    return [isolates_all.index(x) for x in isolates]

def getIvalue(drug,delta=0.005):
    scaler = MinMaxScaler()
    df=pd.read_table('/mounts/data/proj/asgari/dissertation/git_repos/Geno2Pheno/data_config/before_march_2018/Final_MICs_16.06.16.txt')
    isolates_all=df['Isolates'].values.tolist()
    res=df[['Isolates','CIP MIC','TOB MIC','COL MIC','CAZ MIC','MEM MIC']]    
    R_S=df[['Unnamed: 2','Unnamed: 4','Unnamed: 6','Unnamed: 8','Unnamed: 10']].as_matrix().tolist()
    resistances_before=np.array([[float(str(x).replace('<=','').replace('≤','').replace('<=','').replace('≥','').replace('>=','')) for x in row] for row in res[['CIP MIC','TOB MIC','COL MIC','CAZ MIC','MEM MIC']].as_matrix()])
    resistances_after=scaler.fit_transform(resistances_before)
    isolates=[x[0] for idx, x in enumerate(list(df[['Isolates']].values))]
    drugs=['CIP','TOB','COL','CAZ','MEM']
    drugs_name=['Ciprofloxacin','Tobramycin','Colistin','Ceftazidim','Meropenem']
    idx=drugs.index(drug)
    
    ###
    I_isolates=df[df['Unnamed: '+str(idx*2+2)]=='I']['Isolates'].values.tolist()
    I_idxs=getIDXs(I_isolates,df)
    mean_I=np.mean(resistances_after[I_idxs,idx])
    
    
    #plt.hist(resistances_after[:,idx], bins='auto')  
    #plt.title("Histogram with 'auto' bins")
    #plt.show()
    ###
    
    dfmiss=pd.read_table('miscl_all.txt')
    dfmiss=dfmiss[dfmiss['mode']=='expr']
    dfmiss=dfmiss[dfmiss['drug']==drugs_name[idx]]
    
    miss_classified=list(dfmiss['sample'].values)
    miss_idxs=getIDXs(miss_classified,df)
    miss_idxs_close = np.where(np.abs(resistances_after[miss_idxs,idx]-mean_I)<delta)[0].tolist()
    miss_idxs_close = [miss_idxs[i] for i in miss_idxs_close]
    miss_idxs_not_close = list(set(miss_idxs) - set(miss_idxs_close))
    
    all_idxs=list(range(len(isolates_all)))
    all_idxs=list(set(all_idxs) - set(I_idxs))
    corr_idxs=list(set(all_idxs) - set(miss_idxs))

    corr_idxs_close = np.where(np.abs(resistances_after[corr_idxs,idx]-mean_I)<delta)[0].tolist()
    corr_idxs_close = [corr_idxs[i] for i in corr_idxs_close]
    corr_idxs_not_close = list(set(corr_idxs) - set(corr_idxs_close))
    
    contig=[[len(corr_idxs_not_close),len(corr_idxs_close)],[len(miss_idxs_not_close),len(miss_idxs_close)]]
    contig=np.array(contig)
    c, p, dof, expected = chi2_contingency(contig)
    return p, contig


In [261]:

def find_the_best_thr(drug):
    p=1
    t=0
    final_table=[]
    for i in tqdm.tqdm(np.arange(0.001,0.999,0.001)):
        try:
            p_c,tab=getIvalue(drug,delta=i)
            if p_c<p:
                p=p_c
                t=i
                final_table=tab
        except:
            a=1
    return (p,t,final_table) 

In [262]:
results=dict()
for drug in ['CIP','TOB','CAZ','MEM']:
    results[drug]=find_the_best_thr(drug)

  0%|          | 0/998 [00:00<?, ?it/s]/usr/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
100%|██████████| 998/998 [00:15<00:00, 64.13it/s]


In [263]:
results

{'CAZ': (6.146024371639494e-08, 0.001, array([[488,   0],
         [ 61,   5]])), 'CIP': (0.012710574366163874, 0.055, array([[213, 306],
         [ 17,  52]])), 'MEM': (0.0009139829929751118,
  0.001,
  array([[551,   0],
         [ 51,   2]])), 'TOB': (0.295721148607811, 0.006, array([[434, 148],
         [ 27,  14]]))}

In [253]:
final_table

array([[488,   0],
       [ 61,   5]])